In [2]:
# This code is used to store the ML models used for the web app

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, fbeta_score, precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, Perceptron, RidgeClassifier, SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import multilabel_confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from skmultilearn.adapt import MLkNN
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.neural_network import MLPClassifier
     
from nltk.stem.snowball import SnowballStemmer

import pandas as pd
import json
import os
import re

import gensim 
from gensim.models import Word2Vec
import gensim.downloader

import pickle 

import nltk
from nltk.tokenize import word_tokenize

import spacy
import matplotlib
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
from ast import literal_eval
from tqdm import tqdm

import sklearn.metrics
import numpy as np

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /homes/lgf21/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
tqdm.pandas()

# Opening Files: 

In [4]:
# Open Pickle lemmatised for tactics: 

with open('merged_data_lemma.pickle', 'rb') as handle: # opening tactics
    (X_train_text, X_test_text, Y_train, Y_test, _, _) = pickle.load(handle)

In [8]:
Y_test.columns

Index(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005',
       'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'],
      dtype='object')

# Feature Extraction: 

In [5]:
import sys
sys.path.append('../src')

In [6]:
# call method
from feature_extraction import feature_extraction, evaluation

In [9]:
pd.DataFrame([1,2,3,4]).reset_index().apply(lambda row: print(row.index),axis=1)

Index(['index', 0], dtype='object')
Index(['index', 0], dtype='object')
Index(['index', 0], dtype='object')
Index(['index', 0], dtype='object')


0    None
1    None
2    None
3    None
dtype: object

In [21]:
X_train, X_test, fe = feature_extraction('TfIdfVectorizer', X_train_text, X_test_text)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [22]:
# save tf-idf as pickle: 

with open('tfidf_tac_lemma.pickle', 'wb') as handle:
    pickle.dump(fe, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Tactic Model: 

In [25]:
# Train and test: First delete techniques less than 9 
# We fix the random state to have the same dataset in our different tests

sv_classifier = OneVsRestClassifier(LinearSVC(penalty = 'l2', loss = 'squared_hinge', dual = False, max_iter = 1000, class_weight = 'balanced', random_state=42), n_jobs = 1)
sv_classifier.fit(X_train, Y_train)


OneVsRestClassifier(estimator=LinearSVC(class_weight='balanced', dual=False,
                                        random_state=42),
                    n_jobs=1)

In [19]:
Y_pred = pd.DataFrame(sv_classifier.predict(X_test), columns=Y_test.columns)

In [23]:
evaluation(Y_pred, Y_test)

,metric,result
0,macro precision,0.666731
1,micro precision,0.705984
2,macro recall,0.640867
3,micro recall,0.665576
4,macro fscore,0.660490
5,micro fscore,0.697515


In [30]:
# Save as pickle: 

with open('tactic_model_lem.pickle', 'wb') as handle:
    pickle.dump(sv_classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
len(sv_classifier.feature_names_in_)

73887

In [28]:
# Save as pickle: 

with open('tactic_train_test.pickle', 'wb') as handle:
    pickle.dump((X_train, X_test, Y_train, Y_test), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
with open('train_text.pickle', 'wb') as handle:
    pickle.dump(X_train_text, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Technique Model:

In [5]:
# Open Pickle lemmatised for techniques: 

with open('merged_data_lemma.pickle', 'rb') as handle:
    (X_train_text, X_test_text, _, _, Y_train, Y_test) = pickle.load(handle)

In [10]:
X_train, X_test = feature_extraction('TfIdfVectorizer', X_train_text, X_test_text)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [11]:
# Train and test: First delete techniques less than 9 
# We fix the random state to have the same dataset in our different tests@

sv_classifier = OneVsRestClassifier(LinearSVC(penalty = 'l2', loss = 'squared_hinge', dual = False, max_iter = 1000, class_weight = 'balanced', random_state=42), n_jobs = 1)
sv_classifier.fit(X_train, Y_train)


OneVsRestClassifier(estimator=LinearSVC(class_weight='balanced', dual=False,
                                        random_state=42),
                    n_jobs=1)

In [12]:
Y_pred = pd.DataFrame(sv_classifier.predict(X_test), columns=Y_test.columns)

In [13]:
# Save as pickle: 

with open('technique_model_lemma.pickle', 'wb') as handle:
    pickle.dump((X_train, X_test, Y_train, Y_test), handle, protocol=pickle.HIGHEST_PROTOCOL)

OSError: [Errno 122] Disk quota exceeded